# Setup

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
import holidays

import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from src.prepare_data import TSDataset, generate_lags
from src.feature_engineering import (add_holiday, add_day_length, add_temporal, 
                                     add_weekends, add_sun_position, generate_cyclic_features)
from src.model_training import Pipeline


# Prepare Data and Horizons

In [7]:
data = pd.read_csv('../data/electricity_load_data.csv', parse_dates=True, index_col=['datetime'])
data.head()

,load
datetime,
2019-01-01 00:00:00,112.01
2019-01-01 01:00:00,92.44
2019-01-01 02:00:00,84.52
2019-01-01 03:00:00,75.36
2019-01-01 04:00:00,63.64


In [8]:
# Generate target and input variables
lookback = 72
n_lags = lookback - 1
horizon = 24

dataset = TSDataset(dataframe=data, target_variable='load')
X, y = dataset.to_supervised(n_lags=n_lags, horizon=horizon)

In [27]:
y

,load+1,load+2,load+3,load+4,load+5,load+6,load+7,load+8,load+9,load+10,...,load+15,load+16,load+17,load+18,load+19,load+20,load+21,load+22,load+23,load+24
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-03 23:00:00,57.12,51.09,48.97,49.80,49.50,55.26,81.05,105.59,99.36,90.73,...,93.24,107.48,129.92,178.56,186.29,167.09,140.96,119.31,101.16,86.22
2019-01-04 00:00:00,51.09,48.97,49.80,49.50,55.26,81.05,105.59,99.36,90.73,82.60,...,107.48,129.92,178.56,186.29,167.09,140.96,119.31,101.16,86.22,65.87
2019-01-04 01:00:00,48.97,49.80,49.50,55.26,81.05,105.59,99.36,90.73,82.60,88.58,...,129.92,178.56,186.29,167.09,140.96,119.31,101.16,86.22,65.87,57.21
2019-01-04 02:00:00,49.80,49.50,55.26,81.05,105.59,99.36,90.73,82.60,88.58,89.22,...,178.56,186.29,167.09,140.96,119.31,101.16,86.22,65.87,57.21,47.95
2019-01-04 03:00:00,49.50,55.26,81.05,105.59,99.36,90.73,82.60,88.58,89.22,91.60,...,186.29,167.09,140.96,119.31,101.16,86.22,65.87,57.21,47.95,46.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-14 19:00:00,78.10,70.46,59.48,51.71,46.99,44.48,40.32,39.61,40.27,43.66,...,82.56,81.36,76.10,78.84,77.27,87.35,108.45,112.67,109.27,93.25
2022-05-14 20:00:00,70.46,59.48,51.71,46.99,44.48,40.32,39.61,40.27,43.66,61.54,...,81.36,76.10,78.84,77.27,87.35,108.45,112.67,109.27,93.25,80.74
2022-05-14 21:00:00,59.48,51.71,46.99,44.48,40.32,39.61,40.27,43.66,61.54,81.92,...,76.10,78.84,77.27,87.35,108.45,112.67,109.27,93.25,80.74,78.99


In [23]:
test_ratio = 0.2
val_ratio = test_ratio / (1-test_ratio)

# Split set once for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=False)

# Split once more for validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)

DatetimeIndex(['2021-09-11 14:00:00', '2021-09-11 15:00:00',
               '2021-09-11 16:00:00', '2021-09-11 17:00:00',
               '2021-09-11 18:00:00', '2021-09-11 19:00:00',
               '2021-09-11 20:00:00', '2021-09-11 21:00:00',
               '2021-09-11 22:00:00', '2021-09-11 23:00:00',
               ...
               '2022-05-14 14:00:00', '2022-05-14 15:00:00',
               '2022-05-14 16:00:00', '2022-05-14 17:00:00',
               '2022-05-14 18:00:00', '2022-05-14 19:00:00',
               '2022-05-14 20:00:00', '2022-05-14 21:00:00',
               '2022-05-14 22:00:00', '2022-05-14 23:00:00'],
              dtype='datetime64[ns]', name='datetime', length=5890, freq=None)

In [26]:
print(f"Test set runs from: {y_test.index.min()} to {y_test.index.max()}")

Test set runs from: 2021-09-11 14:00:00 to 2022-05-14 23:00:00


## Feature Engineering

In [10]:
# Assign danish holidays
holidays_dk = holidays.DK()
X_train = add_holiday(X_train, holidays_dk)

# Add temporal columns
temporal_features = ['hour', 'day_of_week', 'week', 'month', 'quarter']
periods = [24, 7, 53, 12, 4]
start_nums = [0, 0, 1, 1, 1]
X_train = add_temporal(X_train, temporal_features)

# Encode temporal columns
X_train = generate_cyclic_features(
    dataframe=X_train, col_names=temporal_features, periods=periods, start_nums=start_nums)

# Add weekends
X_train = add_weekends(X_train)

# Solar position
lat = 55.73
lon = 9.58

X_train = add_sun_position(X_train, lon, lat)

# Day lengths
X_train = add_day_length(X_train, lon, lat)

print(X_train.shape)

(17669, 87)


# Train a Multi Output Regression Model with XGBoost

In [11]:
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    objective='reg:squarederror',
    tree_method='hist',
    importance_type='gain'
)

multi_reg = MultiOutputRegressor(xgb_reg).fit(X_train, y_train)

In [12]:
feature_importances = []

for h in range(horizon):
    feat_h = multi_reg.estimators_[h].feature_importances_.argsort()[:3] # take top 3 from each estimator
    col_names = X_train.columns.values[feat_h]
    feature_importances.append([col for col in col_names.tolist()])

In [13]:
set([f for f_list in feature_importances for f in f_list ])

{'cos_quarter',
 'load',
 'load-1',
 'load-10',
 'load-11',
 'load-12',
 'load-13',
 'load-18',
 'load-2',
 'load-26',
 'load-3',
 'load-32',
 'load-33',
 'load-34',
 'load-36',
 'load-37',
 'load-38',
 'load-4',
 'load-42',
 'load-45',
 'load-5',
 'load-51',
 'load-52',
 'load-6',
 'load-63',
 'load-65',
 'load-69',
 'load-7',
 'load-70',
 'load-9',
 'sin_quarter',
 'weekend'}

## Conclusion?

## Run data prep with best features once more

In [14]:
dataset = TSDataset(dataframe=data, target_variable='load')
X, y = dataset.to_supervised(n_lags=n_lags, horizon=horizon)

In [15]:
X = add_temporal(X, ['quarter'])
X = generate_cyclic_features(X, col_names=['quarter'], periods=[4], start_nums=[1])
X = add_weekends(X)

c:\Users\nicol\Documents\Teaching\ml_kursus\venv\lib\site-packages\src\feature_engineering.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[feature] = temporal_features[feature]


In [16]:
# Use Pipeline class to automate data preparation
pipe = Pipeline(inputs=X, targets=y, use_validation=True, test_ratio=0.2)

# Decide which scalers to use
x_scaler, y_scaler = StandardScaler(), StandardScaler()

# Run data through the pipeline
train, val, test, y_scaler = pipe.run(x_scaler, y_scaler)

## Save data

In [17]:
torch.save(train, "../data/train.pt")
torch.save(val, "../data/val.pt")
torch.save(test, "../data/test.pt")
pickle.dump(y_scaler, open("../data/scaler.pkl", "wb"))

# To load uncomment:
# train = torch.load("../data/train.pt")
# val = torch.load("../data/val.pt")
# test = torch.load("../data/test.pt")
# scaler = pickle.load(open('../data/scaler.pkl', 'rb'))

# Baseline

In [18]:
# Split set once for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=False)

# Split once more for validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)

In [19]:
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=6,
    objective='reg:squarederror',
    tree_method='hist',
    importance_type='gain'
)

multi_reg = MultiOutputRegressor(xgb_reg).fit(X_train, y_train)

In [20]:
xgb_preds = multi_reg.predict(X_val)

In [21]:
df_pred = pd.DataFrame(xgb_preds, columns = y_val.columns, index=y_val.index)

In [22]:
print('VALIDATION BASELINE')
print('RMSE:\t', mean_squared_error(y_val, df_pred)**0.5)
print('MAE:\t', mean_absolute_error(y_val, df_pred))
print('MAPE:\t',mean_absolute_percentage_error(y_val, df_pred) * 100)
print('R^2:\t', r2_score(y_val, df_pred))

VALIDATION BASELINE
RMSE:	 8.304237273815021
MAE:	 5.832316999712707
MAPE:	 6.9870914573776295
R^2:	 0.920280339597249
